### Deferred Acceptance

In this worksheet, I'll implement the deferred acceptance algorithm with simulated preference data from the vancouver school board Kindergarten Special Programs matching.  


The data is designed to resemble real preference data for 2019.  For example, the overall demand for programs it accurate. The index numbers are not the same as the ones used by the school board. The capacities at each school are informed estimates based on capacities in a prior year and on subsequent changes in provincial class size regulations. So none of this data should be taken literally. The demand for the different programs is not accurate for example.  The point is to provide a dataset that resembles the one used by the school board to do its matching.


The `augmented` field gives the score that the school board might assign to the students based on a system wide lottery that treats each student equally, and on special characteristics of the student that can depend on the school.  For example, a student might get a higher ranking at a school where the student has an older sibling.

The worksheet goes throught a series of steps.  The data on rankings by student is imported.  It is then ordered by student id and the rank of the school by the corresponding student.  This ordering is important for the algorithm which uses the ordering as part of its calculations.


Data on school capacities is then imported and analyzed a bit.  


The algorithm is given in the last part.  It takes the dataframe associated with the re-ordered ranking data and the dataframe associated with the schools capacities and modifies them with the matching calculations it does.  So if you want to make changes and re-run the algorithm, you'll have to rebuild these dataframes before you do.


To get the full deferred acceptance solution, you need to run the algorithm manually three times (so that any student whose proposal is rejected gets a chance to try again.  They are allowed to give three options, so they may need three chances to try them all.

In [1]:
import pandas as pd 
import os
import IPython
ds = pd.read_csv("anonymized.csv")
ds.sort_values(by ='sIndex')

,Unnamed: 0,First Name,CHOICE,ProgramName,sIndex,open,matched,augmented
0,1894.0,Grayson,1,Montessorri Alternate Program - Renfrew,277479538,True,False,1.668609664437562
2117,1894.0,Grayson,1,Montessorri Alternate Program - Renfrew,277479538,True,False,1.668609664437562
2,669.0,Jeved,3,Early French Immersion - Laura Secord,277731278,True,False,0.7405798479018244
1,1505.0,Jeved,1,Early Mandarin Bilingual - Norquay,277731278,True,False,0.7405798479018244
2118,1505.0,Jeved,1,Early Mandarin Bilingual - Norquay,277731278,True,False,0.7405798479018244
...,...,...,...,...,...,...,...,...
4231,1493.0,Roselynn,3,Early French Immersion - Trafalgar,293785580,True,False,0.026193574934504027
2114,1493.0,Roselynn,3,Early French Immersion - Trafalgar,293785580,True,False,0.026193574934504027
4232,1560.0,Loic,1,Early Mandarin Bilingual - Norquay,293785630,True,False,0.8779472268566664
2115,1560.0,Loic,1,Early Mandarin Bilingual - Norquay,293785630,True,False,0.8779472268566664


We'll leave the raw data as is, and instead make a copy that orders the data by its index number then by the choice.  The ordering over choice is important in the algorithm that uses this ordering to ensure that students make their proposals in the right order. We'll be modifying this new dataframe as we go along to update the `matched` and `open` fields as we create matches and record rejections.

In [14]:
ds_mod = ds.sort_values(['sIndex','CHOICE'], ascending=[False,True])
ds_mod

,Unnamed: 0,First Name,CHOICE,ProgramName,sIndex,open,matched,augmented
2116,NaN,First Name,CHOICE,ProgramName,sIndex,open,matched,augmented
2115,1560.0,Loic,1,Early Mandarin Bilingual - Norquay,293785630,True,False,0.8779472268566664
4232,1560.0,Loic,1,Early Mandarin Bilingual - Norquay,293785630,True,False,0.8779472268566664
2113,323.0,Roselynn,1,Early French Immersion - Jules Quesnel/Queen E...,293785580,True,False,1.0261935749345041
4230,323.0,Roselynn,1,Early French Immersion - Jules Quesnel/Queen E...,293785580,True,False,1.0261935749345041
2114,1493.0,Roselynn,3,Early French Immersion - Trafalgar,293785580,True,False,0.026193574934504027
4231,1493.0,Roselynn,3,Early French Immersion - Trafalgar,293785580,True,False,0.026193574934504027
2111,1802.0,Freya,1,Montessorri Alternate Program - Maple Grove,293767942,True,False,0.006012682146833925
4228,1802.0,Freya,1,Montessorri Alternate Program - Maple Grove,293767942,True,False,0.006012682146833925
2110,873.0,Freya,2,Early French Immersion - L'Ecole Bilingue,293767942,True,False,0.006012682146833925


Next, how many students are involved in the special programs draw.

In [15]:
#for index,element in ds_mod.iterrows():
 #   print(element['sIndex'],element['augmented'],element['ProgramName'])
ds['sIndex'].nunique()

934

There are 18 schools, each school has potentially a different number of seats.  We will read the capacity from a file below.  Concisely, the list of schools looks like this:

In [4]:
school_list = ds['ProgramName'].unique().tolist()
school_list

['Montessorri Alternate Program - Renfrew',
 'Early Mandarin Bilingual - Norquay',
 'Early French Immersion - Laura Secord',
 'Early French Immersion - Selkirk',
 'Early French Immersion - Jules Quesnel/Queen Elizabeth Annex',
 'Early French Immersion - Hastings',
 'Fine Arts - Nootka',
 'Montessorri Alternate Program - Tyee',
 'Early French Immersion - Douglas Annex',
 'Montessorri Alternate Program - Maple Grove',
 "Early French Immersion - L'Ecole Bilingue",
 'Early French Immersion - Tennyson',
 'Early French Immersion - Hudson',
 'Early French Immersion - Quilchena',
 'Early French Immersion - Trafalgar',
 'Early French Immersion - Kerrisdale',
 'Early French Immersion - Strathcona',
 "Indigenous Focus - Xpey'",
 'ProgramName']

The next step is to read in the data about schools.  This illustrates how to read information in a json file, which is much better than csv.

In [5]:
import json
with open("schools_data.json", "r") as read_file:
    schools = json.load(read_file)

schools

{'Early French Immersion - Douglas Annex': {'members': [], 'seats': 60},
 'Early French Immersion - Hastings': {'members': [], 'seats': 40},
 'Early French Immersion - Hudson': {'members': [], 'seats': 20},
 'Early French Immersion - Jules Quesnel/Queen Elizabeth Annex': {'members': [],
  'seats': 40},
 'Early French Immersion - Kerrisdale': {'members': [], 'seats': 40},
 "Early French Immersion - L'Ecole Bilingue": {'members': [], 'seats': 60},
 'Early French Immersion - Laura Secord': {'members': [], 'seats': 40},
 'Early French Immersion - Quilchena': {'members': [], 'seats': 20},
 'Early French Immersion - Selkirk': {'members': [], 'seats': 40},
 'Early French Immersion - Strathcona': {'members': [], 'seats': 20},
 'Early French Immersion - Tennyson': {'members': [], 'seats': 60},
 'Early French Immersion - Trafalgar': {'members': [], 'seats': 40},
 'Early Mandarin Bilingual - Norquay': {'members': [], 'seats': 20},
 'Fine Arts - Nootka': {'members': [], 'seats': 20},
 "Indigenous 

Like the dataframe, ds_mod, we can modify this dictionary as we go along by adding students id numbers to the members array.

In [6]:
sum = 0
for school in schools:
    sum += schools[school]['seats']
    
sum

600

So there are 933 families competing for 600 seats, still a large excess demand.


The next bit is a program that runs one round of the deferred acceptance algorithm. The first part is a function that decides whether or not to accept a student proposal.  If the school has already filled all its seats, then the function will check whether the rating of the student exceeds the rating of the worst student the school has already accepted.


The program itself starts on line 53 (assuming you enabled line numbers).  Since each applicant has a maximum of three choices, you can run the full algorithm by exectuing the program three times in succession.

In [9]:
# sIndex
def proposal(sIndex,augmented,ProgramName,ds_mod,schools):
    """
    A student represented by sIndex proposes to a school called ProgramName.
    sIndex - a numeric id number for a student, links to a specific row in the ds dataframe
    augmented - the students rank used to decide whether to displace existing applicants
    ProgramName - the text string associated with a school
    ds - the dataframe associated with the students preferences
    schools - a dictionary of school names mapped to arrays with student index numbers(sIndex)
    return - True if the schools is currently willing to accept a student, False otherwise.
    ****
    The routine checks if the school currently has space, if so it appends the student id to its
    member list, changes the matching field to True in the ds data frame and returns true.  
    If not, it checks to see whether the applicant is better than the worst student
    in its list.  If so, it removes the worst student from its member list, changes the open
    field in the data frame ds for the student to False, adds the new applicant to its members
    list, and changes the matching field for the new applicant to true, then returns true.
    Otherwise it switches the open fields in the data frame to false and returns false.
    """ 
    # first a sanity check - is it there already
    list = {}
    if sIndex in schools[ProgramName]['members']:
      
        return False
        #print("already there")
           
    
    # is there room? if yes add it, otherwise go on
    if len(schools[ProgramName]['members']) < schools[ProgramName]['seats']:
       # print(schools[ProgramName]['seats'])
        #return
        schools[ProgramName]['members'].append(sIndex)
        ds_mod.loc[(ds_mod['ProgramName'] == ProgramName)&(ds_mod['sIndex'] == sIndex),['matched']]=True
        return True
    #create a list of current members with rankings
    for spin in schools[ProgramName]['members'] :
        f = ds_mod.loc[(ds_mod['ProgramName'] == ProgramName)&(ds_mod['sIndex'] == spin)]
        list[spin] =f.iloc[0]['augmented'] 
        del f
    #get the minimum value in the new list
    v = min(list,key=list.get)
    if augmented <= list[v]:
        #reject
        ds_mod.loc[(ds_mod['ProgramName'] == ProgramName)&(ds_mod['sIndex'] == sIndex),['open']]=False
        return False
    #remove the old one
    ds_mod.loc[(ds_mod['ProgramName'] == ProgramName)&(ds_mod['sIndex'] == v),['open']]=False
    ds_mod.loc[(ds_mod['ProgramName'] == ProgramName)&(ds_mod['sIndex'] == v),['matched']]=False
    schools[ProgramName]['members'].remove(v)
    # add the new one
    schools[ProgramName]['members'].append(sIndex)
    ds_mod.loc[(ds_mod['ProgramName'] == ProgramName)&(ds_mod['sIndex'] == sIndex),['matched']]=True
    return True
current = 0
for index,element in ds_mod.iterrows():
    if element['matched'] == False and current != element['sIndex'] and element['open'] == True:
        test = proposal(element['sIndex'],element['augmented'],element['ProgramName'],ds_mod,schools)
        current = element['sIndex']  
    elif element['matched'] == True:
        current = element['sIndex']
    else:
        continue
dss = ds_mod[ds_mod['matched'] == True]
dss

,Unnamed: 0,First Name,CHOICE,ProgramName,sIndex,open,matched,augmented


The next display just shows the modified dataframe with the matching results.  When the column value `matched` is True, the deferred acceptance algorithm has admitted the corresponding student into the school.

In [10]:
ds_mod

,Unnamed: 0,First Name,CHOICE,ProgramName,sIndex,open,matched,augmented
2116,NaN,First Name,CHOICE,ProgramName,sIndex,open,matched,augmented
2115,1560.0,Loic,1,Early Mandarin Bilingual - Norquay,293785630,True,False,0.8779472268566664
4232,1560.0,Loic,1,Early Mandarin Bilingual - Norquay,293785630,True,False,0.8779472268566664
2113,323.0,Roselynn,1,Early French Immersion - Jules Quesnel/Queen E...,293785580,True,False,1.0261935749345041
4230,323.0,Roselynn,1,Early French Immersion - Jules Quesnel/Queen E...,293785580,True,False,1.0261935749345041
2114,1493.0,Roselynn,3,Early French Immersion - Trafalgar,293785580,True,False,0.026193574934504027
4231,1493.0,Roselynn,3,Early French Immersion - Trafalgar,293785580,True,False,0.026193574934504027
2111,1802.0,Freya,1,Montessorri Alternate Program - Maple Grove,293767942,True,False,0.006012682146833925
4228,1802.0,Freya,1,Montessorri Alternate Program - Maple Grove,293767942,True,False,0.006012682146833925
2110,873.0,Freya,2,Early French Immersion - L'Ecole Bilingue,293767942,True,False,0.006012682146833925


Now we can focus on the dataframe consisting of all the matched applicants.  The next calculation shows the very high proportion of applicants who were matched to their first choice (481 of them). Only 63 got their second choice, 40 their third choice.

In [11]:
dst = ds_mod[ds_mod.matched == True]
dst['CHOICE'].value_counts()


Series([], Name: CHOICE, dtype: int64)

Not all of the schools are filled to capacity.  This is a consequence of the fact that each family is only allowed to specify 3 choices.  The next list shows each school, the numer of seats it has, then the number of applicants it was finally given.

In [12]:
for school in schools:
    print(school,schools[school]['seats'],len(schools[school]['members']))

Early French Immersion - Selkirk 40 0
Early French Immersion - Tennyson 60 0
Early French Immersion - Hastings 40 0
Fine Arts - Nootka 20 0
Early French Immersion - Strathcona 20 0
Montessorri Alternate Program - Maple Grove 20 0
Early Mandarin Bilingual - Norquay 20 0
Montessorri Alternate Program - Renfrew 20 0
Early French Immersion - Laura Secord 40 0
Early French Immersion - Jules Quesnel/Queen Elizabeth Annex 40 0
Early French Immersion - Kerrisdale 40 0
Indigenous Focus - Xpey' 20 0
Early French Immersion - Trafalgar 40 0
Early French Immersion - Quilchena 20 0
Montessorri Alternate Program - Tyee 20 0
Early French Immersion - Hudson 20 0
Early French Immersion - L'Ecole Bilingue 60 0
Early French Immersion - Douglas Annex 60 0
